In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://butlerdata-group2-project3.s3.us-east-2.amazonaws.com/Tweets.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Tweets.csv"), sep=",", header=True)

# Show DataFrame
df.show(60)

+--------------------+-----------------+----------------------------+--------------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            tweet_id|airline_sentiment|airline_sentiment_confidence|      negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|         tweet_coord|       tweet_created|      tweet_location|       user_timezone|
+--------------------+-----------------+----------------------------+--------------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  570306133677760513|          neutral|                         1.0| 

In [ ]:
df = df.filter(df.airline != "null")

df.show(60)

+------------------+-----------------+----------------------------+--------------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|      negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|         tweet_coord|       tweet_created|      tweet_location|       user_timezone|
+------------------+-----------------+----------------------------+--------------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|570306133677760513|          neutral|                         1.0|         

In [ ]:
from pyspark.sql.functions import length

data_df = df.withColumn('length', length(df['text']))
data_df.show()

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|      tweet_location|       user_timezone|length|
+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+------+
|570306133677760513|          neutral|                         1.0|          null|                  

In [ ]:
# Tokenize DataFrame
tokened = Tokenizer(inputCol="text", outputCol="words")
tokened_transformed = tokened.transform(df)
tokened_transformed.show()

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|      tweet_location|       user_timezone|               words|
+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|570306133677760513|          neutral|                    

In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
removed_frame = remover.transform(tokened_transformed)
removed_frame.show(truncate=False)

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------------+----------------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|tweet_id          |airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|airline       |airline_sentiment_gold|name           |negativereason_gold|retweet_count|text                                                                    

In [ ]:
# Remove stop words
stop_list = ["@united", "@jetblue's", "@VirginAmerica", "@virginamerica","@AmericanAir","@JetBlue","@jetblue","@DeltaAssist","@SouthwestAir","@united","@United","airlines","Airlines","@USAirways","@usairways"]
remover = StopWordsRemover(inputCol="filtered", outputCol="filtered_new", stopWords=stop_list)
removed_frame_new = remover.transform(removed_frame)
removed_frame_new.show(truncate=False)

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------------+----------------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|tweet_id          |airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|airline       |airline_sentiment_gold|name 

In [ ]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="filtered", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(removed_frame_new)
hashed_df.show()

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|      tweet_location|       user_timezone|               words|            filtered|        filtered_new|        hashedValues|
+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+-----------------

In [ ]:
# remove tweet_coord
hashed_clean_df = hashed_df.drop("tweet_coord", "negativereason", "negativereason_confidence","airline_sentiment_gold","negativereason_gold")
hashed_clean_df.count()

14658

In [ ]:
hashed_clean_df = hashed_clean_df.filter(hashed_clean_df.text != "null")
hashed_clean_df.count()

14632

In [ ]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_clean_df)
rescaledData = idfModel.transform(hashed_clean_df)

In [ ]:
# Display the DataFrame
rescaledData.show(truncate=False)

+------------------+-----------------+----------------------------+--------------+---------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+----------------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.feature import OneHotEncoder

In [ ]:
pos_neg_to_num = StringIndexer(inputCol='airline_sentiment',outputCol='label')
removed_frame_new = pos_neg_to_num.fit(rescaledData)
removed_frame_new2 = removed_frame_new.transform(rescaledData)
removed_frame_new2.show()

+------------------+-----------------+----------------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|       airline|           name|retweet_count|                text|       tweet_created|      tweet_location|       user_timezone|               words|            filtered|        filtered_new|        hashedValues|            features|label|
+------------------+-----------------+----------------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|570306133677760513|          neutral|                         1.0|Virgin Am

In [ ]:
onehot = OneHotEncoder(inputCol='label',outputCol='hotcode')
one_hot_test = onehot.fit(removed_frame_new2).transform(removed_frame_new2)
removed_frame_new3 = onehot.fit(removed_frame_new2)
removed_frame_new4 = removed_frame_new3.transform(removed_frame_new2)
# removed_frame_new4.show()
one_hot_test.show()

+------------------+-----------------+----------------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|       airline|           name|retweet_count|                text|       tweet_created|      tweet_location|       user_timezone|               words|            filtered|        filtered_new|        hashedValues|            features|label|      hotcode|
+------------------+-----------------+----------------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+
|570306133677760513|          neut

In [ ]:
cleaned = one_hot_test

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'hotcode']).show()

+-----+-------------+
|label|      hotcode|
+-----+-------------+
|  1.0|(2,[1],[1.0])|
|  2.0|    (2,[],[])|
|  1.0|(2,[1],[1.0])|
|  0.0|(2,[0],[1.0])|
|  0.0|(2,[0],[1.0])|
|  0.0|(2,[0],[1.0])|
|  2.0|    (2,[],[])|
|  1.0|(2,[1],[1.0])|
|  2.0|    (2,[],[])|
|  2.0|    (2,[],[])|
|  1.0|(2,[1],[1.0])|
|  2.0|    (2,[],[])|
|  2.0|    (2,[],[])|
|  2.0|    (2,[],[])|
|  2.0|    (2,[],[])|
|  0.0|(2,[0],[1.0])|
|  2.0|    (2,[],[])|
|  0.0|(2,[0],[1.0])|
|  2.0|    (2,[],[])|
|  2.0|    (2,[],[])|
+-----+-------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
#predictor.save("sentiment_model.h5")

# New Section

In [ ]:
# import cloudpickle
# with open("predictor.pkl", "wb") as fid:
#   cloudpickle.dump(gnb, fid)

In [ ]:
import shutil

In [ ]:
from pyspark.ml.classification import NaiveBayesModel 

In [ ]:
from pyspark.mllib.util import MLUtils
output_dir = 'my_Naives_model'
shutil.rmtree(output_dir, ignore_errors=True)
predictor.save(output_dir)

In [ ]:
model = NaiveBayesModel.load(output_dir)

# New Section

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+------------------+-----------------+----------------------------+---------+-------------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+--------------------+----------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|  airline|         name|retweet_count|                text|       tweet_created| tweet_location|       user_timezone|               words|            filtered|        filtered_new|        hashedValues|            features|label|      hotcode|       rawPrediction|         probability|prediction|
+------------------+-----------------+----------------------------+---------+-------------+-------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.483498


In [ ]:
testing.show()

+------------------+-----------------+----------------------------+----------+---------------+-------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|   airline|           name|retweet_count|                text|       tweet_created|  tweet_location|       user_timezone|               words|            filtered|        filtered_new|        hashedValues|            features|label|      hotcode|
+------------------+-----------------+----------------------------+----------+---------------+-------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+
|567594449874587648|         negative|                    

**Start of Model moving to Pandas and SciKit Learn**

In [ ]:
df = cleaned['airline_sentiment','text']

In [ ]:
import pandas as pd
pd_df = df.toPandas()

In [ ]:
pd_df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,"""@VirginAmerica it's really aggressive to blas..."
4,negative,@VirginAmerica and it's a really big bad thing...


In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
pd_df['sentiment_code'] = lb.fit_transform(pd_df['airline_sentiment'])
pd_df.head()

,airline_sentiment,text,sentiment_code
0,neutral,@VirginAmerica What @dhepburn said.,1
1,positive,@VirginAmerica plus you've added commercials t...,2
2,neutral,@VirginAmerica I didn't today... Must mean I n...,1
3,negative,"""@VirginAmerica it's really aggressive to blas...",0
4,negative,@VirginAmerica and it's a really big bad thing...,0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

# text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB())])

# create the pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words = stop_list)),
                     ('tfidf', TfidfTransformer(use_idf = True)),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

# split the data into test and train
X=pd_df['text']
y=pd_df['sentiment_code']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

text_clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['americanair', 'deltaassist', 'jetblue', 'southwestair', 'united', 'usairways', 'virginamerica'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['@united', "@jetblue's",
                                             '@VirginAmerica', '@virginamerica',
                                             '@AmericanAir', '@JetBlue',
                                             '@j...DeltaAssist',
                                             '@SouthwestAir', '@united',
                                             '@United', 'airlines', 'Airlines',
                                             '@USAirways', '@usairways'],
                         

In [ ]:
# run model with test data
from sklearn.metrics import classification_report

clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test), digits=4))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['americanair', 'deltaassist', 'jetblue', 'southwestair', 'united', 'usairways', 'virginamerica'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['americanair', 'deltaassist', 'jetblue', 'southwestair', 'united', 'usairways', 'virginamerica'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['americanair', 'deltaassist', 'jetblue', 'southwestair', 'united', 'usairways', 'virginamerica'] 

              precision    recall  f1-score   support

           0     0.7742    0.9667    0.8598      3660
           1     0.7496    0.3918    0.5146      1261
           2     0.8125    0.5440    0.6517       932

    accuracy                         0.7755      5853
   macro avg     0.7788    0.6341    0.6753      5853
weighted avg     0.7750    0.7755    0.7523      5853



In [ ]:
# test the model with a sample tweet
var = ["@VirginAmerica it was amazing, and arrived an hour early. You're too good to me"]
clf.predict(var)

array([2])

In [ ]:
# export the model and saved model name
import joblib
joblib.dump(clf,"final_model.sav")

['final_model.sav']